#**Assignment 10 - NLP** | María Daniela Baptista Theuerkauf

In [1]:
!pip install datasets
from datasets import list_datasets
datasets_list = list_datasets()
#print(datasets_list)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1+cu121 requir

<ipython-input-1-5f9a16c8bb6a>:3: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  datasets_list = list_datasets()
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
!pip install pyLDAvis
!pip install rouge_score
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 19.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=ca19a89374ef89bfffeaf70cce4d970612bf03159bbf3692b42f68e5b9c5ba13
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvid

In [3]:
from datasets import load_dataset
from datasets import load_from_disk
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import string
from multiprocessing import Pool, cpu_count
import re
import gensim
from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis
import pyLDAvis.gensim_models
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import BartTokenizer, BartForConditionalGeneration
from datasets import load_metric
from nltk.translate.bleu_score import sentence_bleu
import numpy as np


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

#Loading Dataset

In [4]:
dataset = load_dataset("cnn_dailymail", '3.0.0')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [18]:
print(dataset)
print(dataset['train'][0])
print(dataset['train'][1])
print(dataset['train'][2])

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})
{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#Preprocessing

In [ ]:
# Preprocessing tools
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    emoticon_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        "]+", flags=re.UNICODE)  # Remove emoticons
    text = emoticon_pattern.sub(r'', text)
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation and special characters
    text = re.sub(r'\d+', '', text)  # Remove digits
    tokens = word_tokenize(text)  # Tokenize
    tokens = [word for word in tokens if word not in stop_words]  # Remove stop words
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatize
    return ' '.join(tokens)

def preprocess_batch(batch):
    batch['article'] = [preprocess_text(text) for text in batch['article']]
    batch['highlights'] = [preprocess_text(text) for text in batch['highlights']]
    return batch

# Load the dataset
dataset = load_dataset("cnn_dailymail", '3.0.0')

# Use multiprocessing to preprocess the dataset
num_proc = cpu_count()  # Use the number of available CPU cores

# Preprocess the dataset in parallel
dataset = dataset.map(preprocess_batch, batched=True, batch_size=1000, num_proc=num_proc)
print(dataset['train'][0])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Map (num_proc=2):   0%|          | 0/287113 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/13368 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/11490 [00:00<?, ? examples/s]

{'article': 'london england reuters harry potter star daniel radcliffe gain access reported million million fortune turn monday insists money wont cast spell daniel radcliffe harry potter harry potter order phoenix disappointment gossip columnist around world young actor say plan fritter cash away fast car drink celebrity party dont plan one people soon turn suddenly buy massive sport car collection something similar told australian interviewer earlier month dont think ill particularly extravagant thing like buying thing cost pound book cd dvd radcliffe able gamble casino buy drink pub see horror film hostel part ii currently six place number one movie uk box office chart detail hell mark landmark birthday wrap agent publicist comment plan ill definitely sort party said interview hopefully none reading radcliffes earnings first five potter film held trust fund able touch despite growing fame rich actor say keeping foot firmly ground people always looking say kid star go rail told repor

Saving Preprocessed Dataset

In [5]:
#from google.colab import drive
#drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mounted at /content/drive


In [ ]:
# Save dataset
#dataset.save_to_disk('/content/drive/My Drive/preprocessed_dataset')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mounted at /content/drive


Saving the dataset (0/2 shards):   0%|          | 0/287113 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13368 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11490 [00:00<?, ? examples/s]

Loading Preprocessed Dataset from Drive

In [6]:
# Load preprocessed dataset
#dataset = load_from_disk('/content/drive/My Drive/preprocessed_dataset')

# Verfiying
#print(dataset['train'][0])



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'article': 'london england reuters harry potter star daniel radcliffe gain access reported million million fortune turn monday insists money wont cast spell daniel radcliffe harry potter harry potter order phoenix disappointment gossip columnist around world young actor say plan fritter cash away fast car drink celebrity party dont plan one people soon turn suddenly buy massive sport car collection something similar told australian interviewer earlier month dont think ill particularly extravagant thing like buying thing cost pound book cd dvd radcliffe able gamble casino buy drink pub see horror film hostel part ii currently six place number one movie uk box office chart detail hell mark landmark birthday wrap agent publicist comment plan ill definitely sort party said interview hopefully none reading radcliffes earnings first five potter film held trust fund able touch despite growing fame rich actor say keeping foot firmly ground people always looking say kid star go rail told repor

In [7]:
print(dataset['train'][0])
print(dataset['train'][1])
print(dataset['train'][2])

{'article': 'london england reuters harry potter star daniel radcliffe gain access reported million million fortune turn monday insists money wont cast spell daniel radcliffe harry potter harry potter order phoenix disappointment gossip columnist around world young actor say plan fritter cash away fast car drink celebrity party dont plan one people soon turn suddenly buy massive sport car collection something similar told australian interviewer earlier month dont think ill particularly extravagant thing like buying thing cost pound book cd dvd radcliffe able gamble casino buy drink pub see horror film hostel part ii currently six place number one movie uk box office chart detail hell mark landmark birthday wrap agent publicist comment plan ill definitely sort party said interview hopefully none reading radcliffes earnings first five potter film held trust fund able touch despite growing fame rich actor say keeping foot firmly ground people always looking say kid star go rail told repor

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#Statistics

In [14]:
def calculate_document_statistics(dataset_split, sample_size=1000):
    # Sample the dataset
    sampled_data = dataset_split.select(range(sample_size))

    doc_lengths = []
    token_counts = []
    target_lengths = []
    sentence_lengths = []
    vocab_sizes = []

    for example in sampled_data:
        # Document length
        doc_length = len(example['article'])
        doc_lengths.append(doc_length)

        # Token count
        tokens = word_tokenize(example['article'])
        token_count = len(tokens)
        token_counts.append(token_count)

        # Target length
        target_length = len(example['highlights'])
        target_lengths.append(target_length)

        # Vocabulary size
        vocab_size = len(set(tokens))
        vocab_sizes.append(vocab_size)

    # Convert lists to numpy arrays for easier statistical calculation
    doc_lengths = np.array(doc_lengths)
    token_counts = np.array(token_counts)
    target_lengths = np.array(target_lengths)
    sentence_lengths = np.array(sentence_lengths)
    vocab_sizes = np.array(vocab_sizes)

    # Calculate statistics
    stats = {
        'document_length': {
            'mean': np.mean(doc_lengths),
            'median': np.median(doc_lengths),
            'std_dev': np.std(doc_lengths),
            'min': np.min(doc_lengths),
            'max': np.max(doc_lengths),
        },
        'token_count': {
            'mean': np.mean(token_counts),
            'median': np.median(token_counts),
            'std_dev': np.std(token_counts),
            'min': np.min(token_counts),
            'max': np.max(token_counts),
        },
        'target_length': {
            'mean': np.mean(target_lengths),
            'median': np.median(target_lengths),
            'std_dev': np.std(target_lengths),
            'min': np.min(target_lengths),
            'max': np.max(target_lengths),
        },
        'vocabulary_size': {
            'mean': np.mean(vocab_sizes),
            'median': np.median(vocab_sizes),
            'std_dev': np.std(vocab_sizes),
            'min': np.min(vocab_sizes),
            'max': np.max(vocab_sizes),
        }
    }

    return stats

# Calculate statistics for the training set
stats = calculate_document_statistics(dataset['train'])

# Print statistics
print("Document Length Statistics:")
print(f"Mean: {stats['document_length']['mean']}")
print(f"Median: {stats['document_length']['median']}")
print(f"Standard Deviation: {stats['document_length']['std_dev']}")
print(f"Min: {stats['document_length']['min']}")
print(f"Max: {stats['document_length']['max']}")

print("\nToken Count Statistics:")
print(f"Mean: {stats['token_count']['mean']}")
print(f"Median: {stats['token_count']['median']}")
print(f"Standard Deviation: {stats['token_count']['std_dev']}")
print(f"Min: {stats['token_count']['min']}")
print(f"Max: {stats['token_count']['max']}")

print("\nTarget Length Statistics:")
print(f"Mean: {stats['target_length']['mean']}")
print(f"Median: {stats['target_length']['median']}")
print(f"Standard Deviation: {stats['target_length']['std_dev']}")
print(f"Min: {stats['target_length']['min']}")
print(f"Max: {stats['target_length']['max']}")

print("\nVocabulary Size Statistics:")
print(f"Mean: {stats['vocabulary_size']['mean']}")
print(f"Median: {stats['vocabulary_size']['median']}")
print(f"Standard Deviation: {stats['vocabulary_size']['std_dev']}")
print(f"Min: {stats['vocabulary_size']['min']}")
print(f"Max: {stats['vocabulary_size']['max']}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Document Length Statistics:
Mean: 2391.304
Median: 2226.0
Standard Deviation: 1105.1950513750955
Min: 136
Max: 6652

Token Count Statistics:
Mean: 333.546
Median: 307.5
Standard Deviation: 155.7379975600046
Min: 23
Max: 934

Target Length Statistics:
Mean: 188.986
Median: 188.0
Standard Deviation: 36.28525601397901
Min: 87
Max: 295

Vocabulary Size Statistics:
Mean: 218.411
Median: 207.0
Standard Deviation: 92.52835283846784
Min: 15
Max: 573


#Topic Modeling: LDA

In [ ]:
# Sample a smaller subset of the dataset because I runned out of RAM
sample_size = 180000  # Adjust the sample size as needed
sampled_data = dataset['train'].select(range(sample_size))

# Extract preprocessed articles
articles = sampled_data['article']

# Tokenize the articles
def tokenize(texts):
    return [word_tokenize(text) for text in texts]

tokenized_articles = tokenize(articles)



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Convert the list of tokenized articles to a dictionary and corpus
dictionary = corpora.Dictionary(tokenized_articles)
corpus = [dictionary.doc2bow(text) for text in tokenized_articles]

# Train the LDA model with fewer passes to reduce memory usage
lda_model = LdaModel(corpus, num_topics=10, id2word=dictionary, passes=5)

# Print the topics
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

# Visualize the topics
lda_visualization = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(lda_visualization)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(0, '0.015*"said" + 0.010*"car" + 0.006*"water" + 0.005*"fire" + 0.005*"road" + 0.005*"est" + 0.004*"two" + 0.004*"flight" + 0.004*"one" + 0.004*"driver"')
(1, '0.014*"said" + 0.012*"war" + 0.012*"u" + 0.009*"military" + 0.009*"attack" + 0.008*"soldier" + 0.008*"force" + 0.007*"security" + 0.007*"army" + 0.006*"killed"')
(2, '0.015*"said" + 0.012*"state" + 0.010*"president" + 0.008*"new" + 0.007*"obama" + 0.007*"u" + 0.006*"nh" + 0.006*"would" + 0.005*"tory" + 0.005*"american"')
(3, '0.019*"mr" + 0.018*"said" + 0.011*"minister" + 0.009*"would" + 0.008*"government" + 0.007*"labour" + 0.006*"party" + 0.006*"country" + 0.006*"people" + 0.006*"mp"')
(4, '0.011*"per" + 0.010*"cent" + 0.009*"year" + 0.008*"said" + 0.007*"million" + 0.006*"people" + 0.006*"new" + 0.006*"also" + 0.006*"company" + 0.005*"could"')
(5, '0.007*"one" + 0.006*"like" + 0.006*"show" + 0.005*"new" + 0.005*"time" + 0.005*"year" + 0.004*"said" + 0.004*"video" + 0.004*"first" + 0.004*"scroll"')
(6, '0.009*"game" + 0.008*"

#Text Summarization with T5

In [7]:
# Load T5 model and tokenizer
model_name_t5 = "t5-small"
tokenizer_t5 = T5Tokenizer.from_pretrained(model_name_t5)
model_t5 = T5ForConditionalGeneration.from_pretrained(model_name_t5)

# Summarization function using T5
def t5_summary(text, max_length=150, min_length=30):
    input_text = "summarize: " + text
    inputs = tokenizer_t5.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model_t5.generate(
        inputs,
        max_length=max_length,
        min_length=min_length,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )
    summary = tokenizer_t5.decode(outputs[0], skip_special_tokens=True)
    return summary

# Apply the summarization to the dataset using T5
def compare_summaries_t5(example):
    text = example['article']
    example['t5_summary'] = t5_summary(text)
    return example

# Apply T5 summarization
sample_size = 1000
sampled_data_with_t5 = dataset['train'].select(range(sample_size))
sampled_data_with_t5 = sampled_data_with_t5.map(compare_summaries_t5)

# Print a few examples
for i in range(5):
    print(f"Original Text: {sampled_data_with_t5[i]['article'][:500]}...")
    print(f"T5 Summary: {sampled_data_with_t5[i]['t5_summary']}")
    print("\n")

# Metrics: ROUGE and BLEU for T5
rouge = load_metric("rouge",trust_remote_code=True)
nltk.download('punkt')

def calculate_rouge(predicted_summary, reference_summary):
    scores = rouge.compute(predictions=[predicted_summary], references=[reference_summary])
    return scores['rouge1'].mid.fmeasure, scores['rouge2'].mid.fmeasure, scores['rougeL'].mid.fmeasure

def calculate_bleu(predicted_summary, reference_summary):
    reference_tokens = nltk.word_tokenize(reference_summary)
    predicted_tokens = nltk.word_tokenize(predicted_summary)
    bleu_score = sentence_bleu([reference_tokens], predicted_tokens)
    return bleu_score

# Initialize scores for T5
rouge1_scores_t5 = []
rouge2_scores_t5 = []
rougeL_scores_t5 = []
bleu_scores_t5 = []

# Evaluate T5 summaries
for example in sampled_data_with_t5:
    reference_summary = example['highlights']
    t5_summary_text = example['t5_summary']

    rouge1, rouge2, rougeL = calculate_rouge(t5_summary_text, reference_summary)
    rouge1_scores_t5.append(rouge1)
    rouge2_scores_t5.append(rouge2)
    rougeL_scores_t5.append(rougeL)
    bleu_scores_t5.append(calculate_bleu(t5_summary_text, reference_summary))

# Average the scores for T5
average_rouge1_t5 = sum(rouge1_scores_t5) / len(rouge1_scores_t5)
average_rouge2_t5 = sum(rouge2_scores_t5) / len(rouge2_scores_t5)
average_rougeL_t5 = sum(rougeL_scores_t5) / len(rougeL_scores_t5)
average_bleu_t5 = sum(bleu_scores_t5) / len(bleu_scores_t5)

# Print T5 results
print("Average ROUGE-1 for T5:", average_rouge1_t5)
print("Average ROUGE-2 for T5:", average_rouge2_t5)
print("Average ROUGE-L for T5:", average_rougeL_t5)
print("Average BLEU Score for T5:", average_bleu_t5)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Original Text: london england reuters harry potter star daniel radcliffe gain access reported million million fortune turn monday insists money wont cast spell daniel radcliffe harry potter harry potter order phoenix disappointment gossip columnist around world young actor say plan fritter cash away fast car drink celebrity party dont plan one people soon turn suddenly buy massive sport car collection something similar told australian interviewer earlier month dont think ill particularly extravagant thing like...
T5 Summary: harry potter star daniel radcliffe gain access reported million million fortune turn monday insists money wont cast spell daniel radcliffe harry potter harry potter order phoenix disappointment gossip columnist around world young actor say plan fritter cash away fast car drink celebrity party dont plan one people soon turn suddenly buy massive sport car collection something similar told australian interviewer earlier month dont think ill particularly extravagant th

<ipython-input-7-197a16d922b9>:39: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge",trust_remote_code=True)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram over

Average ROUGE-1 for T5: 0.2679514727237582
Average ROUGE-2 for T5: 0.1014181499919996
Average ROUGE-L for T5: 0.1967458203343604
Average BLEU Score for T5: 0.035797981523044774


In [37]:
#print(sampled_data_with_t5[0].keys())

dict_keys(['article', 'highlights', 'id', 't5_summary'])


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Text Summarization with Bart

In [12]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load BART model and tokenizer
model_name_bart = "facebook/bart-large-cnn"
tokenizer_bart = BartTokenizer.from_pretrained(model_name_bart)
model_bart = BartForConditionalGeneration.from_pretrained(model_name_bart)

# Summarization function using BART
def bart_summary(text, max_length=150, min_length=30, length_penalty=2.0, num_beams=4):
    inputs = tokenizer_bart.encode_plus(
        text,
        return_tensors="pt",
        max_length=1024,
        truncation=True
    )

    summary_ids = model_bart.generate(
        inputs['input_ids'],
        max_length=max_length,
        min_length=min_length,
        length_penalty=length_penalty,
        num_beams=num_beams,
        early_stopping=True
    )

    summary = tokenizer_bart.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Apply the summarization to the dataset using BART
def compare_summaries_bart(example):
    text = example['article']
    example['bart_summary'] = bart_summary(text)
    return example

# Apply BART summarization
sampled_data_with_bart = dataset['train'].select(range(sample_size))
sampled_data_with_bart = sampled_data_with_bart.map(compare_summaries_bart)

# Print a few examples to check
for i in range(5):
    print(f"Original Text: {sampled_data_with_bart[i]['article'][:500]}...")
    print(f"BART Summary: {sampled_data_with_bart[i]['bart_summary']}")
    print("\n")

# Metrics: ROUGE and BLEU for BART
rouge1_scores_bart = []
rouge2_scores_bart = []
rougeL_scores_bart = []
bleu_scores_bart = []

# Evaluate BART summaries
for example in sampled_data_with_bart:
    reference_summary = example['highlights']
    bart_summary_text = example['bart_summary']

    rouge1, rouge2, rougeL = calculate_rouge(bart_summary_text, reference_summary)
    rouge1_scores_bart.append(rouge1)
    rouge2_scores_bart.append(rouge2)
    rougeL_scores_bart.append(rougeL)
    bleu_scores_bart.append(calculate_bleu(bart_summary_text, reference_summary))

# Average the scores for BART
average_rouge1_bart = sum(rouge1_scores_bart) / len(rouge1_scores_bart)
average_rouge2_bart = sum(rouge2_scores_bart) / len(rouge2_scores_bart)
average_rougeL_bart = sum(rougeL_scores_bart) / len(rougeL_scores_bart)
average_bleu_bart = sum(bleu_scores_bart) / len(bleu_scores_bart)

# Print BART results
print("Average ROUGE-1 for BART:", average_rouge1_bart)
print("Average ROUGE-2 for BART:", average_rouge2_bart)
print("Average ROUGE-L for BART:", average_rougeL_bart)
print("Average BLEU Score for BART:", average_bleu_bart)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Original Text: london england reuters harry potter star daniel radcliffe gain access reported million million fortune turn monday insists money wont cast spell daniel radcliffe harry potter harry potter order phoenix disappointment gossip columnist around world young actor say plan fritter cash away fast car drink celebrity party dont plan one people soon turn suddenly buy massive sport car collection something similar told australian interviewer earlier month dont think ill particularly extravagant thing like...
BART Summary: london england reuters harry potter star daniel radcliffe gain access reported million million fortune turn monday insists money wont cast spell. young actor say plan fritter cash away fast car drink celebrity party dont plan one people soon turn suddenly buy massive sport car collection.


Original Text: editor note behind scene series cnn correspondent share experience covering news analyze story behind event soledad obrien take user inside jail many inmate men

In [39]:
#print(sampled_data[0].keys())

dict_keys(['article', 'highlights', 'id', 'bart_summary'])


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#Comparison Metrics: T5 and BART

In [13]:
# Print comparison between T5 and BART
print("\nComparison between T5 and BART:")
print(f"ROUGE-1: T5={average_rouge1_t5}, BART={average_rouge1_bart}")
print(f"ROUGE-2: T5={average_rouge2_t5}, BART={average_rouge2_bart}")
print(f"ROUGE-L: T5={average_rougeL_t5}, BART={average_rougeL_bart}")
print(f"BLEU: T5={average_bleu_t5}, BART={average_bleu_bart}")



Comparison between T5 and BART:
ROUGE-1: T5=0.2679514727237582, BART=0.27158309390239144
ROUGE-2: T5=0.1014181499919996, BART=0.0903802358468822
ROUGE-L: T5=0.1967458203343604, BART=0.190283350504156
BLEU: T5=0.035797981523044774, BART=0.024779915612841093


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
